In [9]:
# imports
import requests
import json

import open3d as o3d
import numpy as np

# get the id of the first annotation
url = "http://localhost:4322/api/queries/findAnnotation"
response = requests.get(url)
id = json.loads(response.text)[0]
print(id)

3d87d707-5735-4d7d-a401-679ae2a748b5


In [10]:
tgt_points_id = "3d87d707-5735-4d7d-a401-679ae2a748b5"
src_points_id = "4b854809-1601-4e7e-aa78-85aa19fa43d3"

In [13]:
# Define the URL and query parameters
url = "http://localhost:4322/api/annotations/3d87d707-5735-4d7d-a401-679ae2a748b5/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
tgt_points = np.array(json.loads(response.text))

print(f"points: {tgt_points}")

# Define the URL and query parameters
url = "http://localhost:4321/api/annotations/4b854809-1601-4e7e-aa78-85aa19fa43d3/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
src_points = np.array(json.loads(response.text))

print(f"points: {src_points}")


points: [[137.15062074 -56.11408989 -29.09428223]
 [108.25235213 -46.44734016 -27.29873521]
 [ 72.28858124 -58.32665047 -23.85722113]
 [  4.47920139 -91.81704426 -42.33971145]]
points: [[ 701757.85653848 3142014.52990013 1073527.84141308]
 [ 701746.07743941 3142024.38444733 1073498.90227216]
 [ 701763.76885274 3142030.26883148 1073463.93951211]
 [ 701813.23370933 3142061.36943788 1073394.22420414]]


In [14]:
# Convert numpy arrays to Open3D PointClouds
source_pcd = o3d.geometry.PointCloud()
source_pcd.points = o3d.utility.Vector3dVector(src_points)

target_pcd = o3d.geometry.PointCloud()
target_pcd.points = o3d.utility.Vector3dVector(tgt_points)

In [19]:
# Define correspondences as index pairs
correspondences = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])

correspondences_o3d = o3d.utility.Vector2iVector(correspondences)

# Create a TransformationEstimationPointToPoint object
estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

# Compute the rigid transformation matrix
transformation_matrix = estimation.compute_transformation(target_pcd, source_pcd, correspondences_o3d)

# Print the transformation matrix
print("Computed Transformation Matrix:")
print(transformation_matrix)

Computed Transformation Matrix:
[[-9.55238971e-02 -3.07990171e-01  9.46581872e-01  1.87031379e+04]
 [-9.57203477e-01  2.89405582e-01 -2.43162464e-03 -2.35037772e+05]
 [-2.73197160e-01 -9.06303737e-01 -3.22454412e-01  3.38547244e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# GREETING TEST

# Define the URL and query parameters
url = "http://localhost:4322/api/exp/greet/Orti"
url = "http://localhost:4322/api/exp/greet"
params = {"name": "Orti"}

# Send a GET request to the Suave endpoint
response = requests.get(url, params=params)

# Check the response
if response.status_code == 200:
    print(f"Response: {response.text}")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")

In [ ]:
url = "http://localhost:4321/api/queries/queryAnnotationAsObj"
payload = {
    "annotationId": id, 
    "queryAttributes":[], 
    "distanceToPlane": 100000.0,
    "outputReferenceFrame": "global",
    "outputGeometryType": "mesh"
}
print(payload)
response = requests.post(url, json=payload)
response.raise_for_status() 
filename = "./6952.obj"
file = open(filename, "w")
file.write(response.text)
file.close()